In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

from config import ACCESS_KEY,SECRET_KEY,TOKEN,APP_METRICA_OAUTH

In [2]:
def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    #now = datetime.datetime.strptime('2024-09-01 00:00:00','%Y-%m-%d %H:%M:%S')
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    days_to_subtract = yesterday.day - 1
    start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
    yesterday = now - datetime.timedelta(days=1)
    end_of_previous_month = start_of_month_data - datetime.timedelta(days=1)
    days_to_subtract_previous_month = end_of_previous_month.day - 1
    start_of_previous_month = end_of_previous_month - datetime.timedelta(days=days_to_subtract_previous_month)
    for_last_month = yesterday - relativedelta(months=1)
    return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'now_date':now.strftime('%Y-%m-%d'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d'),
            'start_of_previous_month':start_of_previous_month.strftime('%Y-%m-%d'),
            'end_of_previous_month':end_of_previous_month.strftime('%Y-%m-%d'),
            'last_month':for_last_month.strftime('%m'),
            'last_month_year':for_last_month.strftime('%Y')
            }

# Переменные, которые могут быть использованы в разоичных запросах. 
now = get_now_datetime_str()['now']
# Даты ниже отражают вчерашний день
yesterday = get_now_datetime_str()['yesterday']
yesterday_data = get_now_datetime_str()['yesterday_data']
day = get_now_datetime_str()['day'] # 
month = get_now_datetime_str()['month']
year = get_now_datetime_str()['year']
start_of_month_data = get_now_datetime_str()['start_of_month_data']
now_date = get_now_datetime_str()['now_date']
start_of_previous_month = get_now_datetime_str()['start_of_previous_month']
end_of_previous_month = get_now_datetime_str()['end_of_previous_month']
last_month = get_now_datetime_str()['last_month']
last_month_year = get_now_datetime_str()['last_month_year']

limit = 100
offset = 1




In [3]:
headers={'Authorization':APP_METRICA_OAUTH,'Host':'api.appmetrica.yandex.ru'}
limit = 100
offset = 1


def get_request_metrica(api_html_metrica):
    response = requests.get(
        api_html_metrica,
        headers = headers
    )
    return response

In [4]:
api_html_appmetrica_auditorium = open('api_html_appmetrica_auditorium.txt','r').read().format(start_of_month_data,yesterday_data,offset,limit)
response = get_request_metrica(api_html_appmetrica_auditorium)

In [5]:
# Преобразование файлов из 

api_html_appmetrica_auditorium = open('api_html_appmetrica_auditorium.txt','r').read().format(start_of_month_data,yesterday_data,offset,limit)
response = get_request_metrica(api_html_appmetrica_auditorium)

for i in range(0,len(response.json()['data'])):
    if i == 0:
        data_DAU = pd.DataFrame({
            'Дата': response.json()['data'][i]['dimensions'][0]['name'],
            'Пользователи': [int(response.json()['data'][i]['metrics'][0])],
            'Новые пользователи':  [int(response.json()['data'][i]['metrics'][2])],
            'Доля новых пользователей': [response.json()['data'][i]['metrics'][4]]
            })
    else:
        data_DAU_new_row = pd.DataFrame({
            'Дата': response.json()['data'][i]['dimensions'][0]['name'],
            'Пользователи': [int(response.json()['data'][i]['metrics'][0])],
            'Новые пользователи':  [int(response.json()['data'][i]['metrics'][2])],
            'Доля новых пользователей': [response.json()['data'][i]['metrics'][4]]
            })
        data_DAU = pd.concat([data_DAU,data_DAU_new_row])

data_DAU.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/DAU/{year}/{int(month)}.csv', index=False, sep=';')

data_MAU = pd.DataFrame({
    'Пользователи': [int(response.json()['totals'][0])],
    'Новые пользователи':  [int(response.json()['totals'][2])],
    'Доля новых пользователей': [response.json()['totals'][4]]
    })

data_MAU.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/MAU/{year}/{int(month)}.csv', index=False, sep=';')

In [8]:
# appmetrica_ID_count
api_html_appmetrica_ID_count = open('api_html_appmetrica_ID_count.txt','r').read().format('2022-01-01',yesterday_data,offset,limit)
api_html_appmetrica_ID_count = open('api_html_appmetrica_ID_count.txt','r').read().format('2021-01-01',yesterday_data,offset,limit)
api_html_appmetrica_ID_count = open('api_html_appmetrica_ID_count.txt','r').read().format('2020-01-01',yesterday_data,offset,limit)
api_html_appmetrica_ID_count = open('api_html_appmetrica_ID_count.txt','r').read().format('2019-01-01',yesterday_data,offset,limit)
response = get_request_metrica(api_html_appmetrica_ID_count)

id_count = pd.DataFrame({
    'Android': int(response.json()['data'][0]['metrics'][0]),
    'Ios': int(response.json()['data'][1]['metrics'][0])
    }
    ,index=[0])

id_count
id_count.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_ID_count/remade/{int(year)}/{int(month)}.csv', index=False, sep=";")

In [9]:
# appmetrica_installetion_count

api_html_appmetrica_installetion_count = open('api_html_appmetrica_installetion_count.txt','r').read().format('2019-01-01',yesterday_data,limit,offset)
response = get_request_metrica(api_html_appmetrica_installetion_count)

installetion_count = pd.DataFrame({
    'Все установки': int(response.json()['totals'][1]),
    'Сессии': int(response.json()['totals'][4])
    }
    ,index=[0])

installetion_count.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_all_daownloaded/remade/{int(year)}/{int(month)}.csv', index=False,sep=";")

In [10]:
# appmetrica_installetion_count_per_day

api_html_appmetrica_installetion_count_per_day = open('api_html_appmetrica_installetion_count_per_day.txt','r').read().format(start_of_month_data,yesterday_data)
response = get_request_metrica(api_html_appmetrica_installetion_count_per_day)
response = response.text

list_of_downloads_date = []
for i in json.loads(response)['time_intervals']:
    list_of_downloads_date.append(i[0])

list_of_downloads_count = []
for i in json.loads(response)['totals'][0]:
    list_of_downloads_count.append(i)

count_per_day = pd.DataFrame({
            'Дата': list_of_downloads_date,
            'Установки': list_of_downloads_count
            })

count_per_day.to_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_download_by_day/remade/{int(year)}/{int(month)}.csv', index=False,sep=";")

In [11]:
token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3
BUCKET_NAME = 'dwh-asgard' #имя бакета
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "temp_file"

In [12]:
def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'



def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j)}',"
                else:
                    special_str = f"{special_str}{str(j)},"
            temp_file.write(special_str[:-1]+'\n') 
        offset +=1000 # увеличивает смещение

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

In [13]:
FOLDER = 'total_active_users_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(now_date,start_of_month_data,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [14]:
FOLDER = 'active_users_in_flats_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(now_date,start_of_month_data,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [15]:
FOLDER = 'total_account_count_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [16]:
FOLDER = 'app_in_flats_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [17]:
FOLDER = 'flats_in_sputnic_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [18]:
FOLDER = 'paying_active_flats_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [19]:
FOLDER = 'mobile_report_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data,month,year)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [20]:
FOLDER = 'service_payments_rep_mobule'

service_payments = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(last_month)}.csv')
service_payments.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")

In [21]:
FOLDER = 'family_rate_rep_mobile'

service_payments = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(last_month)}.csv')
service_payments.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")

In [22]:
FOLDER = 'DAF_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [23]:
FOLDER = 'MAF_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [24]:
FOLDER = 'activated_after_unsubsribtion_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data,start_of_previous_month,end_of_previous_month)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [25]:
FOLDER = 'total_active_users_per_day_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(now_date,start_of_month_data,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [26]:
FOLDER = 'total_active_users_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(now_date,start_of_month_data,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [27]:
FOLDER = 'total_activated_account_rep_mobile'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()


In [28]:
FOLDER = 'subscriptions_report_comerce_1_69_rep_mobile_2'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data, now_date, yesterday_data, start_of_previous_month, end_of_previous_month, int(month), int(year), int(last_month), int(last_month_year))

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [29]:
FOLDER = 'subscriptions_report_comerce_1_69_rep_mobile_1'
offset = 0

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data, now_date, yesterday_data, start_of_previous_month, end_of_previous_month, int(month), int(year), int(last_month), int(last_month_year))

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [30]:
FOLDER = 'subscriptions_report_comerce_rep_mobile'
offset = 0
query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data,year,month)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [31]:
FOLDER = 'all_flats_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

In [32]:
# def get_now_datetime_str(): # получаем актуальное время
#     time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
#     #now = datetime.datetime.strptime('2024-04-01 00:00:00','%Y-%m-%d %H:%M:%S')
#     #now = datetime.datetime.now(pytz.timezone(time_zone))    
#     yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
#     days_to_subtract = yesterday.day - 1
#     start_of_month_data = yesterday - datetime.timedelta(days=days_to_subtract)
#     yesterday = now - datetime.timedelta(days=1)
#     end_of_previous_month = start_of_month_data - datetime.timedelta(days=1)
#     days_to_subtract_previous_month = end_of_previous_month.day - 1
#     start_of_previous_month = end_of_previous_month - datetime.timedelta(days=days_to_subtract_previous_month)
#     for_last_month = yesterday - relativedelta(months=1)
#     return {'now':now.strftime('%Y-%m-%d %H:%M:%S'),
#             'now_date':now.strftime('%Y-%m-%d'),
#             'yesterday_data':yesterday.strftime('%Y-%m-%d'),
#             'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
#             'year':yesterday.strftime('%Y'),
#             'month':yesterday.strftime('%m'),
#             'day':yesterday.strftime('%d'),
#             'start_of_month_data':start_of_month_data.strftime('%Y-%m-%d'),
#             'start_of_previous_month':start_of_previous_month.strftime('%Y-%m-%d'),
#             'end_of_previous_month':end_of_previous_month.strftime('%Y-%m-%d'),
#             'last_month':for_last_month.strftime('%m'),
#             'last_month_year':for_last_month.strftime('%Y')
#             }

# # Переменные, которые могут быть использованы в разоичных запросах. 
# now = get_now_datetime_str()['now']
# # Даты ниже отражают вчерашний день
# yesterday = get_now_datetime_str()['yesterday']
# yesterday_data = get_now_datetime_str()['yesterday_data']
# day = get_now_datetime_str()['day'] # 
# month = get_now_datetime_str()['month']
# year = get_now_datetime_str()['year']
# start_of_month_data = get_now_datetime_str()['start_of_month_data']
# now_date = get_now_datetime_str()['now_date']
# start_of_previous_month = get_now_datetime_str()['start_of_previous_month']
# end_of_previous_month = get_now_datetime_str()['end_of_previous_month']
# last_month = get_now_datetime_str()['last_month']
# last_month_year = get_now_datetime_str()['last_month_year']

# limit = 100
# offset = 1

In [33]:
print(f'now: {now}')
print(f'now_date: {now_date}')
print(f'yesterday: {yesterday}')
print(f'yesterday_data: {yesterday_data}')
print(f'start_of_month_data: {start_of_month_data}')

print(f'day: {day}')
print(f'month: {month}')
print(f'year: {year}')

print(f'start_of_previous_month: {start_of_previous_month}')
print(f'end_of_previous_month: {end_of_previous_month}')
print(f'last_month: {last_month}')
print(f'last_month_year: {last_month_year}')


now: 2024-12-01 14:56:36
now_date: 2024-12-01
yesterday: 2024-11-30 14:56:36
yesterday_data: 2024-11-30
start_of_month_data: 2024-11-01
day: 30
month: 11
year: 2024
start_of_previous_month: 2024-10-01
end_of_previous_month: 2024-10-31
last_month: 10
last_month_year: 2024


In [34]:
FOLDER = 'Units_on_SK platform_(monetization)_rep_mobile'

query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
query_description = 'тестовый запрос'#описание, которое появляется в запросах
query_text = open(f'{FOLDER}.txt','r').read().format(start_of_month_data,now_date,yesterday_data)

request_id = create_query()
while str(get_request(0)) == '<Response [400]>':     
    time.sleep(10)

write_temp_file()
temp_file_pd = pd.read_csv(TEMP_FILENAME, sep=',')
temp_file_pd.to_csv(f'C:/Users/Boris/Desktop/mobile rep/{FOLDER}/{int(year)}/{int(month)}.csv', index=False,sep=",")
remove_temp_files()

# Concat

In [35]:
id_count = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_ID_count/remade/{int(year)}/{int(month)}.csv', sep=';')
id_count = id_count[['Android','Ios']]
id_count.rename(columns = {'Android':'Android_id', 'Ios':'iOS_id' },inplace=True)
id_count

,Android_id,iOS_id
0,1907590,1181650


In [36]:
dau = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/DAU/{int(year)}/{int(month)}.csv', sep=';')
dau['DAU']= dau['Пользователи'].mean().round().astype(int)
dau = dau.loc[0,['DAU']]
dau = pd.DataFrame(data = dau.values, columns = dau.index)
dau

,DAU
0,271111


In [37]:
mau = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_DAU_MAU/remade/MAU/{int(year)}/{int(month)}.csv', sep=';')
mau['MAU'] = mau['Пользователи']
mau = mau[['MAU', 'Новые пользователи', 'Доля новых пользователей']]
mau

,MAU,Новые пользователи,Доля новых пользователей
0,839133,74795,8.913367


In [38]:
installetion_count = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/app_metrica_all_daownloaded/remade/{int(year)}/{int(month)}.csv', sep=';')
installetion_count = installetion_count[['Все установки', 'Сессии']]
installetion_count

,Все установки,Сессии
0,2617836,602308592


In [39]:
service_payments = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/service_payments_rep_mobule/{int(year)}/{int(month)}.csv', sep=',')
service_payments = service_payments['service_payments'].to_frame(name = 'service_payments')
service_payments

,service_payments
0,0


In [40]:
mobile_report = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/mobile_report_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
mobile_report

,Первая дата,Месяц,Год,IOS_PL,appstore_count_69,appstore_count_69_refunded,appstore_count_499,appstore_count_499_refunded,appstore_count_2390,appstore_count_2390_refunded,...,yookassa_count_35_refunded,yookassa_count_1,yookassa_count_1_refunded,yookassa_count_499,yookassa_count_499_refunded,yookassa_count_249,yookassa_count_249_refunded,yookassa_count_2390,yookassa_count_2390_refunded,refunded_amount_yookassa
0,'2024-11-01',11,2024,228524,3312,1,0,0,0,0,...,0,7654,113,0,0,0,0,0,0,839


In [41]:
subscriptions_report_comerce = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/subscriptions_report_comerce_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
#subscriptions_report_comerce = subscriptions_report_comerce[['failed_subscriptions_renewed','canceled_subscriptions_renewed','paying_users','paying_users_standart','paying_users_standart_yakassa','paying_users_standart_appstore','paying_users_standart_ios_from_cart','new_subscriptions','failed_subscriptions','canceled_subscriptions','renewed_subscriptions']]
subscriptions_report_comerce

,paying_users,paying_users_standart,paying_users_partner_pro,paying_users_partner_start,paying_users_standart_yakassa1,paying_users_standart_yakassa,paying_users_standart_appstore,paying_users_standart_ios_from_cart,renewed_subscriptions,new_subscriptions,new_subscriptions_activated,failed_subscriptions,failed_subscriptions_renewed,canceled_subscriptions,canceled_subscriptions_renewed
0,52139,52139,32688,17419,0,30688,2910,16937,44596,8244,7543,3947,715,788,739


In [42]:
units_on_SK_platform = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/Units_on_SK platform_(monetization)_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
units_on_SK_platform

,Units on SK platform,Units stricted monetization,Units free monetization,Units_free_monetization_pro,Units_free_monetization_start
0,27512,8081,19431,12221,12221


In [43]:
total_account_count = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/total_account_count_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
total_account_count

,total_account_count
0,1804577


In [44]:
total_activated_account = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/total_activated_account_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
total_activated_account

,total_activated_account,total_activated_account_monetization,total_activated_account_ble_available_monetization,total_activated_account_ble_available
0,1259983,827839,585118,928089


In [45]:
total_active_users = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/total_active_users_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
total_active_users

,total_active_users,total_active_users_monetization,total_active_user_subscribed_monetization,total_active_users_ble_available,total_active_users_ble_available_monetization,total_active_users_ble_available_subscribed_monetization
0,744429,487482,51416,596000,382167,42671


In [46]:
total_active_users_per_day = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/total_active_users_per_day_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
total_active_users_per_day


,total_active_users_per_day,total_active_user_monetization_per_day,total_active_user_subscribed_monetization_per_day
0,237485,148941,18142


In [47]:
all_flats = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/all_flats_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
all_flats

,all_flats,all_flats_partner_pro,all_flats_partner_start,freemonetization_flats,stricted_flats,subsc_restrict_is_null_flats
0,824628,338721,149603,564457,260171,0


In [48]:
flats_in_sputnic = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/flats_in_sputnic_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
flats_in_sputnic

,flats_in_sputnic
0,650573


In [49]:
maf = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/MAF_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
maf

,MAF,stricted_MAF,freemonetization_MAF
0,469606,158008,311598


In [50]:
daf = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/DAF_rep_mobile/{int(year)}/{int(month)}.csv', sep=',').round().astype('int')
daf

,DAF,stricted_DAF,freemonetization_DAF
0,187240,67554,119686


In [51]:
paying_active_flats = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/paying_active_flats_rep_mobile/{int(year)}/{int(month)}.csv', sep=',')
paying_active_flats

,paying_active_flats
0,49821


In [52]:
family_rate_rep_mobile = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/family_rate_rep_mobile/{int(year)}/{int(month)}.csv')
family_rate_rep_mobile

,family_rate_rep_mobile
0,8356


In [53]:
subscriptions_report_comerce_1_69_rep_mobile_1 = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/subscriptions_report_comerce_1_69_rep_mobile_1/{int(year)}/{int(month)}.csv')
subscriptions_report_comerce_1_69_rep_mobile_1

,renewed_subscriptions_69,renewed_subscriptions_69_amount_1,renewed_subscriptions_69_amount_69,renewed_subscriptions_1,renewed_subscriptions_1_amount_69,renewed_subscriptions_1_amount_1,deactivated_since_last_month_subscriptions,deactivated_this_month_subscriptions
0,33101,84,33017,10005,10005,0,5976,0


In [54]:
subscriptions_report_comerce_1_69_rep_mobile_2 = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/subscriptions_report_comerce_1_69_rep_mobile_2/{int(year)}/{int(month)}.csv')
subscriptions_report_comerce_1_69_rep_mobile_2

,active_subscriptions,active_subscriptions_extended,active_subscriptions_new_last_month,active_subscriptions_69,active_subscriptions_69_extended,active_subscriptions_69_extended_from_amount_1,active_subscriptions_69_extended_from_amount_69,active_subscriptions_69_new_last_month,active_subscriptions_1,active_subscriptions_1_extended,active_subscriptions_1_extended_from_amount_1,active_subscriptions_1_extended_from_amount_69,active_subscriptions_1_new_last_month
0,52139,43279,8860,44502,43104,10054,33050,1398,7637,175,0,175,7462


In [55]:
activated_after_unsubsribtion_rep_mobile = pd.read_csv(f'C:/Users/Boris/Desktop/mobile rep/activated_after_unsubsribtion_rep_mobile/{int(year)}/{int(month)}.csv')
activated_after_unsubsribtion_rep_mobile



,activated_after_unsubsribtion_less_3,activated_after_unsubsribtion_more_3
0,24,1217


In [56]:
concated = pd.concat([dau,
                      mau, 
                      id_count,
                      installetion_count,
                      service_payments['service_payments'],
                      mobile_report,
                      subscriptions_report_comerce, 
                      units_on_SK_platform,
                      total_account_count,
                      total_activated_account,
                      total_active_users,
                      total_active_users_per_day,
                      all_flats,
                      flats_in_sputnic,
                      maf,
                      daf,
                      paying_active_flats,
                      family_rate_rep_mobile,
                      subscriptions_report_comerce_1_69_rep_mobile_1,
                      subscriptions_report_comerce_1_69_rep_mobile_2,
                      activated_after_unsubsribtion_rep_mobile
                      ]
                     , axis=1)
concated

,DAU,MAU,Новые пользователи,Доля новых пользователей,Android_id,iOS_id,Все установки,Сессии,service_payments,Первая дата,...,active_subscriptions_69_extended_from_amount_1,active_subscriptions_69_extended_from_amount_69,active_subscriptions_69_new_last_month,active_subscriptions_1,active_subscriptions_1_extended,active_subscriptions_1_extended_from_amount_1,active_subscriptions_1_extended_from_amount_69,active_subscriptions_1_new_last_month,activated_after_unsubsribtion_less_3,activated_after_unsubsribtion_more_3
0,271111,839133,74795,8.913367,1907590,1181650,2617836,602308592,0,'2024-11-01',...,10054,33050,1398,7637,175,0,175,7462,24,1217


In [57]:
concated.to_csv(f'C:/Users/Boris/Desktop/mobile rep/Ready report/{int(year)}/{int(month)}.csv', sep=',', index=False)

In [58]:

TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow")

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )
    
def upload_dump_to_s3():
    get_s3_instance().upload_file(
        Filename= f'C:/Users/Boris/Desktop/mobile rep/Ready report/{int(year)}/{int(month)}.csv',
        Bucket=BUCKET_NAME,
        Key=f"mobile_report_comerce_2/year={str(year)}/{str(month)}.csv"
    )

In [ ]:
get_s3_instance()
upload_dump_to_s3()

: 